# Tutorial #1: APIを用いたeLabFTWのノート作成

<b>Note:</b> ご不明な点がございましたら、lee.jemyung@naist.ac.jp にご連絡ください

## **Step 7:** APIを使って画像ファイルを入れてみましょう
#### JPG、PNG、PDF、GIFなどの画像ファイルを作成した電子ノートに追加するサンプルコード

#### Note: _Step 7_ で<span style='color: orange'>新しくお知らせする部分のタイトル</span>はオレンジ色で表示し、その他の部分では _Step 1~6_ と同じ内容となります。

## 前準備
###　「.env」ファイルの作成
「.env」ファイルの作成に関しては[**<ins>こちら</ins>**](https://github.com/naist-eln/eln/blob/main/API/eLabFTW_API_env.ipynb)をご覧ください。<br>
「.env」ファイルがすでに用意されている場合は、そのまま次に進んでください。

### <span style='color: orange'>更新実験ノートとテーブルの指定</span>
修正したい実験ノートのIDをここであらかじめ指定するか、下記の「実験ノートリストの確認」からセクションでIDリストを確認して指定する。</br>
指定しない場合、最新の実験ノートを更新することになる。

In [8]:
# experiment IDを指定します。
# 指定せずに空欄にしておく場合、最新のノートを更新することになります。

file_sort_by_name = True    # For the folder reading mode: sort files in the folder by name (if 'image-path' is a folder, or folders)
file_sort_by_time = False   # For the folder reading mode: sort files in the folder by modification time (if 'image-path' is a folder, or folders)
replace_image_files = True  # While uploading new image files, remove the previous files (with the identical names) from the experiment note


experiment_id = ''   

image_extensions = ['jpg', 'jpeg', 'png', 'gif']    # set the image files' extensions. 
default_font_size = '14pt'                          # set a default font size for the case of adding sections.
default_image_pos = 'bottom'                        # set a default image position in the case of no designation

# The following cases illustrate the available image files uploading and displaying modes.
# See the descriptions that accompany each case.
# 'image-position': 'top', 'bottom', '2r3c' (2nd row, 3rd column), or '2r', '3c', 'cTITLE' (place all images in the 2nd row, 3rd column, or column entitled 'TITLE')

properties = {
    # Case no.1: This shows how you can designate the target section, ex.'目的', the uploading file, and detailed styles.
    #            You can fix the image width or height (this keeps the image's ratio), or both. Scaling the image size is also available.
    #            You can omit details of the image or border styles, default settings will be applied.
    '目的': [
        {
        'image-path' : './test_file/images/test_no1.jpg',
        'image-scale': 0.2,
        'image-width' : '',
        'image-height' : '',
        'image-position' : 'bottom',
        'image-align' : 'center',
        'border-width' : '',
        'border-style' : '',
        'border-color' : '',
        },
        {
        'image-path' : './test_file/images/test_no1.png',
        'image-scale': '',
        'image-width' : 200,
        'image-height' : '',
        'image-position' : 'bottom',
        'image-align' : 'right',
        'border-width' : '1px',
        'border-style' : 'solid',
        'border-color' : 'grey',
        },
        {
        'image-path' : './test_file/images/test_no1.gif',
        'image-scale': '',
        'image-width' : 100,
        'image-height' : 100,
        'image-position' : 'bottom',
        'image-align' : 'left',
        'border-width' : '3px',
        'border-style' : 'dashed',
        'border-color' : 'green',
        },
    ],    

    # Case no.2: This shows the single file input, multiple files input, and folder input modes.
    #            Each corresponds to the (table insert) '3r2c'->specific position, '4c' or 'cMethod'->specific column, 
    #                                    (section insert) 'top'-> just after the section title, and 'bottom'-> the end of the section. 
    '目的組成': [
        {
        'image-path' : './test_file/images/test_no4.jpg',
        'image-width' : 100,
        'image-position' : '3r2c',
        },
        {
        'image-path' : ['./test_file/images/test_no1.jpg','./test_file/images/test_no2.jpg','./test_file/images/test_no3.jpg'],
        'image-height' : 100,
        'image-position' : '4c',
        'image-align' : 'center',
        'border-width' : '1px',
        },
        {
        'image-path' : './test_file/images/folder/',
        'image-height' : 100,
        'image-position' : 'cMethod',
        'image-align' : 'right',
        },
        {
        'image-path' : './test_file/images/test_no1.jpg',
        'image-width' : 100,
        'image-height' : 80,
        'image-position' : 'top',
        },
        {
        'image-path' : './test_file/images/test_no2.jpg',
        'image-width' : 100,
        'image-height' : 80,
        'image-position' : 'bottom',
        },
    ],

    # Case no.3: You can create a section by designating a new section title. 
    #            This inserts the new section at the end of the experiment note.
    #            You can use a mix of both file and folder input modes.
    'Appendix:': [
        {
        'image-path' : ['./test_file/images/test_no4.jpg','./test_file/images/folder/'],
        'image-scale': '',
        'image-width' : 80,
        'image-height' : '',
        'image-position' : '',
        'image-align' : '',
        'border-width' : '',
        'border-style' : '',
        'border-color' : '',
        }
    ],
}


### コード駆動準備
ライブラリ、パッケージ、モジュールの呼び出しと設定

In [2]:
import os
import re
import glob
from PIL import Image
import elabapi_python
from elabapi_python.rest import ApiException
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
from dotenv import load_dotenv
import warnings
from urllib3.exceptions import InsecureRequestWarning
from difflib import get_close_matches

# APIクライアントの設定
load_dotenv()
configuration = elabapi_python.Configuration()
configuration.host = os.environ["HOST_URL"]
configuration.api_key['Authorization'] = os.environ["API_KEY"]
configuration.debug = False
configuration.verify_ssl = False

# SSL警告を無効化
warnings.simplefilter('ignore', InsecureRequestWarning)

# 各種のインスタンスを作成
api_client = elabapi_python.ApiClient(configuration)
api_client.set_default_header(header_name = 'Authorization', header_value = configuration.api_key['Authorization'] )
experimentsApi = elabapi_python.ExperimentsApi(api_client)
templatesApi = elabapi_python.ExperimentsTemplatesApi(api_client)
uploadsApi = elabapi_python.UploadsApi(api_client)
itemsApi = elabapi_python.ItemsApi(api_client)

### 実験ノートリストの確認
作成した全実験ノートを出力する。

In [3]:
# 全実験ノートの確認
try:
    # デフォルト設定で実験ノートを取得
    print("デフォルト設定で実験ノートを取得:")
    experimentsList = experimentsApi.read_experiments()
    print(f"Number of experiments: {len(experimentsList)}")
    
    # 必要なフィールドを抽出してリストに格納
    exp_data = []
    for experiment in experimentsList:
        exp_dict = experiment.to_dict()
        exp_data.append({
            '_date': exp_dict.get('_date'),
            'category_title': exp_dict.get('category_title'),
            'category': exp_dict.get('category'),
            'fullname': exp_dict.get('fullname'),
            'id': exp_dict.get('id'),
            'tags': exp_dict.get('tags'),
            'tags_id': exp_dict.get('tags_id'),
            'up_item_id': exp_dict.get('up_item_id')
        })

    # データフレームに変換
    experiments_df = pd.DataFrame(exp_data)
    experiments_df['_date'] = pd.to_datetime(experiments_df['_date'])   # _date列をdatetime型に変換
    experiments_df = experiments_df.sort_values(by='_date', ascending=False)    # 作成日が新しい順にソート
    print(experiments_df)
    
except ApiException as e:
    print("ExperimentsApiを呼び出す際の例外: %s\n" % e)

デフォルト設定で実験ノートを取得:
Number of experiments: 29
        _date category_title category        fullname    id  \
0  2025-03-04           None     None     jemyung lee  2426   
3  2025-02-27           None     None  Shogo Takasuka  2411   
4  2025-02-17           None     None     jemyung lee  2379   
7  2025-01-27           None     None     jemyung lee  2321   
10 2024-12-19           None     None  Shogo Takasuka  2152   
13 2024-12-19           None     None  Shogo Takasuka  2149   
12 2024-12-19           None     None  Shogo Takasuka  2150   
11 2024-12-19           None     None  Shogo Takasuka  2151   
9  2024-12-19           None     None  Shogo Takasuka  2153   
8  2024-12-06           None     None     jemyung lee  2062   
14 2024-11-18           None     None  Shogo Takasuka  1992   
5  2024-11-18           None     None     jemyung lee  1987   
6  2024-11-11           None     None     jemyung lee  1923   
15 2024-11-08           None     None  Shogo Takasuka  1910   
16 2024-11-

### 実験ノートIDの指定
実験ノートリストを確認し、修正するノートのIDを入力する。 </br>
最新のノートを修正する場合は空欄にしておいてもよい。

In [4]:
if not experiment_id:
    experiment_id = ''    # <- ここに修正するノートのIDを記入。 空欄にしておくと最新のノートを修正することになる

### 実験ノートを取得
実験ノートリストから実験ノートを取得する。</br>
指定されたノートIDがある場合は当該ノート情報を取得し、指定されたIDがない場合は最新ノートの情報を取得する。</br>
取得した実験ノートのIDが表示されます。

In [5]:
try:
    experiments_list = experimentsApi.read_experiments()
    print(f"Number of experiments: {len(experiments_list)}")
    
    if experiment_id:
        exp_lst = [str(x.id) for x in experiments_list]
        
        if not experiments_list:
            raise ValueError(f"There's no experiment, number of experiments: {len(experiments_list)}")
        elif experiment_id in exp_lst:
            print(f"Selected experiment ID: {experiment_id}")
        else:
            raise ValueError(f"{experiment_id} does not exist in the experiments list")
    else:
        # 最も新しい実験ノートを取得
        if experiments_list:
            experiments_list = sorted(experiments_list, key=lambda x: x._date, reverse=True)  # 作成日でソート
            experiment_id = experiments_list[0].id  # 最新の実験ノートのIDを取得
            print(f"Latest experiment ID: {experiment_id}")        
        else:
            print("No experiments found.")
            exit()

except ApiException as e:
    print(f"Exception when calling ExperimentsApi->read_experiments: {e}\n")
    exit()

Number of experiments: 29
Selected experiment ID: 2426


## <span style='color: orange'>実験ノートの内容更新</span>

### <span style='color: orange'>ノート更新の準備</span>
テーブルに列を追加する関数とテキストだけを抽出する関数を定義

In [6]:
def addEmptyRow(table, soup, n_row = 1, tr_style = '', td_style = ''):
# Add 'n_row' empty row(s) in the transferred 'table' of 'soup', 
# reflecting the row('tr_style') and cell('td_style') styles.

    r_soup = table.find_all('tr')
    n_cell = len(r_soup[0].find_all('td'))

    if not tr_style:
        tr_style = r_soup[2]['style'] if r_soup[2].attrs.get('style') else ''
    if not td_style:
        td_style = r_soup[2].td['style']

    for i in range(n_row):
        empt_tr = soup.new_tag('tr', style = tr_style) if tr_style else soup.new_tag('tr')
    
        for j in range(n_cell):
            nw_td = soup.new_tag('td', style = td_style)
            nw_td.string = ' '
            empt_tr.append(nw_td)
            
        table.append(empt_tr)
    
    return table

def adjustTable(table, soup, img_pos):
# resize the table to fit the outermost image's position, img_pos: [row, column]
# This returns the resized table's rows(soup), if the original one is smaller than the outermost position,
# Else, this returns the original table's.

    try:
        col_titles = None
        rows = table.find_all('tr')
        n_rows = len(rows) - 2

        if n_rows > 0:
            col_titles = list(map(lambda x:x.text.lower(), rows[1].find_all('td')))   # Column titles (English ver.)
            n_cols = len(col_titles)    
        else:
            raise ValueError(f'Insufficient rows: size {len(rows)}')
        
        # Add insufficient row(s)
        if img_pos[0] > n_rows:
            addEmptyRow(table, soup, img_pos[0]-n_rows)
            rows = table.find_all('tr') 
            n_rows = len(rows) - 2

        # in case of insufficient columns
        if img_pos[1] > n_cols:
            raise ValueError(f"Insufficient columns: image position ({img_pos[1]}) > number of columns ({n_cols})")
        
        return rows, n_rows, n_cols, col_titles
    except Exception as e:
        print(f"An error occurred: {e}\n")
    
def removeSpChar(str, char):
# Remove special characters from the string 'str'
# Can define more removing character(s) by adding it at 'char'.

    try:
        rv_str = re.sub(r'[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', '', str)
        if char: rv_str = rv_str.lstrip(char).strip()
        return rv_str
    except Exception as e:
        print(f"An error occurred: {e}\n")
        
def getSectionSoup(soup_dict, title, ignore_null = False):
# Return a table's 'soup' that follows the 'title' section
# If no sections match, it returns the most similar section's table soup

    soup_title = ''
    if title in soup_dict:
        soup_title = title
    else:
        candidate = get_close_matches(title, list(soup_dict.keys()), 3, 0.8)
        if candidate:
            soup_title = candidate[0]
            print(f"Section {title} was not found: section {soup_title} substitutes for it.")
        elif not ignore_null:
            raise ValueError(f"Section {title} was not found, and there's no candidate.")
    
    if soup_title:    
        return soup_dict[soup_title]
    else:
        return None

def imageFileChecker(img_path, order_by_name = False, order_by_date = False):
# Check if the image file(s) or folder(s) is(are) valid
# It returns a list of image files, or raise errors if there's no image file (in the folder or file path)
# Recursive option for the folder's file glob mode is turned off currently. (You can turn on manually)
# 'order_by_name': sort files in the folder by name (if 'image-path' is a folder, or folders)
# 'order_by_date': sort files in the folder by modification time (if 'image-path' is a folder, or folders)

    img_file_list = []
    
    if isinstance(img_path, list):
        img_path_list = img_path
    else:
        img_path_list = [img_path]
    for img_pth in img_path_list:
        if os.path.exists(img_pth):
            if os.path.isfile(img_pth):
                img_file_list.append(img_pth)
            elif os.path.isdir(img_pth):
                if not img_pth.endswith('/'):
                    img_pth += '/'
                for ext in image_extensions:
                    fl = glob.glob(img_pth + '*.%s' % ext, recursive=True)
                    if order_by_date:
                        fl = sorted(fl, key=os.path.getmtime)
                    if order_by_name:
                        fl = sorted(fl)
                    img_file_list.extend(fl) 
            else:
                raise ValueError(f'No image file nor folder.')
        else:
            raise ValueError(f"File of folder {img_path} does not exist") 
    if img_file_list:
        return img_file_list
    else:
        raise ValueError(f"Empty file list: no image file in the list")

def identifyImagePos(img_pos, ignore_null = True):
# return a list of the image position
# 'ignore_null=True': this returns a default position in the case of the given position is NULL or incorrect

    pos = img_pos.lower() if img_pos else None
    if not pos:
        return default_image_pos if ignore_null else None
    if pos == 'top':
        return ['t']
    elif pos == 'bottom':
        return ['b']
    elif len(pos) == 2 and pos[1] == 'r':
        return ['r', int(pos[0])]
    elif len(pos) == 2 and pos[1] == 'c':
        return ['c', int(pos[0])]
    elif len(pos) == 4 and all(x in ['r', 'c'] for x in [pos[1], pos[3]]):
        if pos[1] == 'r' and pos[3] == 'c':
            return ['rc', [int(pos[0]), int(pos[2])]]
        elif pos[1] == 'c' and pos[3] == 'r':
            return ['rc', [int(pos[2]), int(pos[0])]]
        else:
            raise ValueError(f"Invalid image position: '{img_pos}'")
    elif len(pos) > 1 and pos[0] == 'c':
        return ['ct', img_pos[1:].lower()]
    elif ignore_null:
        return default_image_pos 
    else:
        raise ValueError(f"Invalid image position: '{img_pos}'")
    
def identifyPropValue(prop_dict, prop_key, isnumber = False, null_val = None):
    
    p_val = prop_dict[prop_key] if prop_key in prop_dict else null_val
    if isnumber and isinstance(p_val, str) and p_val:
        p_val = float(p_val)
        
    return p_val
        

### <span style='color: orange'>画像ファイルのアップロード</span>
テーブルからデータが記録されている 3 番目の列からの形式を修正します。<br>
フォント、文字サイズ、文字色、文字位置、テーブル線の太さ、線色、線の形、背景色などを更新するコードです。<br>
「properties」で「all」に該当する値はすべてのテーブルに適用され、特定の項目が指定されていると、該当項目のテーブルだけが修正されます。

In [9]:
file_visible = True         # Option for indicating the files' list corresponding to each section

image_properties = ['image-width', 'image-height', 'image-scale']
border_properties = ['border-width', '1pt'], ['border-style', 'solid'], ['border-color','black']

try:
    experiment = experimentsApi.get_experiment(experiment_id)
    exp_body = experiment.body
    exp_title = experiment.title

    # Parsing the enl body using BS4
    soup = BeautifulSoup(exp_body, 'html.parser')

    cnt_soup = soup.find_all(['h1','p'])    
    cnt_dict = dict()
    
    old_img_tags = soup.find_all('img')
    
    for cs in cnt_soup:
        cst = removeSpChar(cs.text, 'TS')
        if cst:
            cnt_dict[cst] = cs

    for es in properties:
        pr_list = properties[es]
        es_soup = getSectionSoup(cnt_dict, es, ignore_null = True)
        
        # append a new 'h1' tag if it doesn't exist
        if not es_soup:
            es_soup = soup.new_tag('h1', style=f'font-size:{default_font_size}')
            es_soup.string = es
            soup.append(es_soup)

        # Adding image file(s) in the designated place(s)
        for pr in pr_list:
            img_lst = imageFileChecker(pr['image-path'], order_by_name=file_sort_by_name, order_by_date=file_sort_by_time)
            n_img = len(img_lst)

            if file_visible:
                print(f"Given path: {es}, {pr['image-path']}\nDetected file(s): {img_lst}")
                
            pr_img_w, pr_img_h, pr_img_scl = [identifyPropValue(pr, item, isnumber=True) for item in image_properties]
            pr_img_pos = identifyImagePos(pr['image-position'] if 'image-position' in pr else None)

            pr_img_sty = ''
            if 'border-width' in pr or 'border-style' in pr or 'border-color' in pr:
                pr_br_w, pr_br_sty, pr_br_clr = [identifyPropValue(pr, item[0], null_val = item[1]) for item in border_properties]
                pr_img_sty = f'border:{pr_br_w} {pr_br_sty} {pr_br_clr}'
            
            tag_lst = list()    # list of image tag that corresponds to each image file.
            
            # Process for each image file
            for img_pth in img_lst:
                img_name = img_pth.rsplit('/',1)[-1]
                
                # upload image files
                uploadsApi.post_upload('experiments', experiment_id, file=img_pth, comment='Uploaded with APIv2')
                uploads = uploadsApi.read_uploads('experiments', experiment_id)
                upload = list(filter(lambda x: x.real_name == img_name, uploads))[0]
                u_id, u_fname, u_lname = upload.id, upload.real_name, upload.long_name
                
                # determine the image size
                im = Image.open(img_pth)
                img_w, img_h = im.size
                wh_r = img_w/img_h
                
                if pr_img_w:
                    img_w = pr_img_w
                    if pr_img_h:
                        img_h = pr_img_h
                    else:
                        img_h = int(img_w / wh_r)
                elif pr_img_h:
                    img_h = pr_img_h
                    img_w = int(img_h * wh_r)
                elif pr_img_scl:
                    img_w = int(img_w * pr_img_scl)
                    img_h = int(img_h * pr_img_scl)
                
                # determine the image option tag                
                img_tag = soup.new_tag('img', height = f'{img_h}', width = f'{img_w}', alt = f'{u_fname}', src = f'app/download.php?name={u_fname}&f={u_lname}')
                if pr_img_sty:
                  img_tag['style'] = pr_img_sty  

                p_tag = soup.new_tag('p')
                if 'image-align' in pr: 
                    p_tag['style'] = f'text-align:{pr['image-align']}'
                    
                p_tag.append(img_tag)
                tag_lst.append(p_tag)  
                
                # delete the old files that corresponds to the uploaded ones
                if replace_image_files:
                    pr_h_tag = es_soup if es_soup.name == 'h1' else es_soup.find_previous('h1')
                    for tag in filter(lambda x: x.find_previous('h1') is pr_h_tag, old_img_tags):
                        f_name = re.search('\\?name=(.*)&f=', tag['src']).group(1)
                        if u_fname == f_name:
                            tag.extract()
                            print(f'An image that corresponds to the file {f_name} has been removed')
            
            next_soup = es_soup.find_next_sibling(es_soup.name)
            
            if pr_img_pos[0] == 'b' and next_soup:
                for tag in tag_lst:
                    next_soup.insert_before(tag)
                
            elif pr_img_pos[0] == 't' or (pr_img_pos[0] == 'b' and not next_soup):
                for tag in reversed(tag_lst):
                    es_soup.insert_after(tag)
                
            elif pr_img_pos[0] in ['r', 'c', 'rc', 'ct']:
                tb_soup = es_soup.find_next_sibling('table')
                if tb_soup:
                    # img_pos: position of an image in a table (ex. 3rd row, 2nd column -> [3,2])
                    # img_idxs: index of an image in the table codes (ex. 3rd row, 2nd column -> [3-1+2,2-1], adding 2 for the row index is to consider two title rows)
                    
                    if pr_img_pos[0] == 'rc':
                        img_pos = pr_img_pos[1]
                        r_soup, n_rows, n_cols, col_titles = adjustTable(tb_soup, soup, img_pos)
                        img_idxs = [img_pos[0]+1, img_pos[1]-1]
                        # insert images at the designated position
                        cell_soup = r_soup[img_idxs[0]].find_all('td')
                        for tag in tag_lst:
                            cell_soup[img_idxs[1]].append(tag)
                    elif pr_img_pos[0] in ['c', 'ct']:                        
                        col_pos = pr_img_pos[1] if pr_img_pos[0] == 'c' else 0
                        r_soup, n_rows, n_cols, col_titles = adjustTable(tb_soup, soup, [n_img, col_pos])
                        col_idx = col_pos - 1 if pr_img_pos[0] == 'c' else col_titles.index(pr_img_pos[1])
                        
                        # insert each image at the designated column
                        for i, p_tag in enumerate(tag_lst): 
                            r_soup[i+2].find_all('td')[col_idx].append(p_tag)
                    elif pr_img_pos[0] == 'r':
                        row_pos = pr_img_pos[1]
                        r_soup, n_rows, n_cols, col_titles = adjustTable(tb_soup, soup, [row_pos, n_img])
                        row_idx = row_pos + 1

                        # insert each image at the designated column
                        c_soup = r_soup[row_idx].find_all('td')
                        for i, p_tag in enumerate(tag_lst): 
                            c_soup[i].append(p_tag)                        
                else:
                    raise ValueError(f"Section '{es}' has no table.")
                
    update_body = {
        "body": str(soup),
    }

    # update the enl
    update_response = experimentsApi.patch_experiment(experiment_id, body=update_body)
    print(f"Experiment {experiment_id} updated successfully.")
    # print(update_response)
except ApiException as e:
    print(f"Exception when calling ExperimentsApi->patch_experiment: {e}\n")
    if e.body:
        print(f"Error details: {e.body}\n")
except Exception as e:
    print(f"An error occurred: {e}\n")

Given path: 目的, ./test_file/images/test_no1.jpg
Detected file(s): ['./test_file/images/test_no1.jpg']
Given path: 目的, ./test_file/images/test_no1.png
Detected file(s): ['./test_file/images/test_no1.png']
Given path: 目的, ./test_file/images/test_no1.gif
Detected file(s): ['./test_file/images/test_no1.gif']
Given path: 目的組成, ./test_file/images/test_no4.jpg
Detected file(s): ['./test_file/images/test_no4.jpg']
Given path: 目的組成, ['./test_file/images/test_no1.jpg', './test_file/images/test_no2.jpg', './test_file/images/test_no3.jpg']
Detected file(s): ['./test_file/images/test_no1.jpg', './test_file/images/test_no2.jpg', './test_file/images/test_no3.jpg']
Given path: 目的組成, ./test_file/images/folder/
Detected file(s): ['./test_file/images/folder/test_no5.jpg', './test_file/images/folder/test_no6.jpg', './test_file/images/folder/test_no7.jpg']
Given path: 目的組成, ./test_file/images/test_no1.jpg
Detected file(s): ['./test_file/images/test_no1.jpg']
Given path: 目的組成, ./test_file/images/test_no2.jp

### <span style='color: orange'>未使用の画像ファイルの削除</span>
実験ノート更新後に、ノートから除外された（使用しない）画像ファイルをアップロードファイルから削除する。

In [10]:
remove_redundant = True    # remove duplicated files uploaded prior (Warning: it can make errors if the files are listed in the elab note.)
results_visible = True      # Display the deleted files' file names and corresponding comments 

try:
    experiment = experimentsApi.get_experiment(experiment_id)
    exp_body = experiment.body
    exp_title = experiment.title
    
    # Parsing the enl body using BS4
    soup = BeautifulSoup(exp_body, 'html.parser')
    
    # Extracting all long names of uploaded image files
    lname_list = [tag['src'].rsplit('&f=',1)[-1] for tag in soup.find_all('img')]
    
    cnt = 0     # counts the number of deleted files
    
    if remove_redundant:
        for upload in uploadsApi.read_uploads('experiments', experiment_id):
            u_lname, u_rname, u_comment = upload.long_name, upload.real_name, upload.comment
            if not u_lname in lname_list and u_rname.rsplit('.',1)[-1] in image_extensions and u_comment == 'Uploaded with APIv2':
                uploadsApi.delete_upload('experiments', experiment_id, upload.id)
                cnt += 1
                if results_visible:
                    print(f'Deleted: {u_rname}, \'{u_comment}\'')
                    
    print(f"{cnt} redundant file(s) in the experiment {experiment_id} have been removed.")

except ApiException as e:
    print(f"Exception when calling ExperimentsApi->patch_experiment: {e}\n")
    if e.body:
        print(f"Error details: {e.body}\n")
except Exception as e:
    print(f"An error occurred: {e}\n")

Deleted: test_no1.png, 'Uploaded with APIv2'
Deleted: test_no1.pdf, 'Uploaded with APIv2'
Deleted: test_no1.jpg, 'Uploaded with APIv2'
Deleted: test_no7.jpg, 'Uploaded with APIv2'
Deleted: test_no6.jpg, 'Uploaded with APIv2'
Deleted: test_no5.jpg, 'Uploaded with APIv2'
Deleted: test_no4.jpg, 'Uploaded with APIv2'
Deleted: test_no7.jpg, 'Uploaded with APIv2'
Deleted: test_no1.jpg, 'Uploaded with APIv2'
Deleted: test_no2.jpg, 'Uploaded with APIv2'
Deleted: test_no6.jpg, 'Uploaded with APIv2'
Deleted: test_no5.jpg, 'Uploaded with APIv2'
Deleted: test_no3.jpg, 'Uploaded with APIv2'
Deleted: test_no2.jpg, 'Uploaded with APIv2'
Deleted: test_no1.jpg, 'Uploaded with APIv2'
Deleted: test_no4.jpg, 'Uploaded with APIv2'
Deleted: test_no1.jpg, 'Uploaded with APIv2'
Deleted: test_no3.jpg, 'Uploaded with APIv2'
Deleted: test_no2.jpg, 'Uploaded with APIv2'
19 redundant file(s) in the experiment 2426 have been removed.


お疲れ様でした。<br>
更新した電子ラボノートを確認して、画像ファイルが正しくアップロードされているか確認してみましょう。